# Appendix 01 - Data preprocessing: *Literature dates*

## Literature Monsoon Onset and Withdrawal normal dates
Normal dates for monsoon onset and withdrawal are taken form the article by PAI et al 2020. In this script the dates are gridded and a xarray daset is created with the onset and withdrawal dates.


In [7]:
# Required Imports
import numpy as np
import pandas as pd
import geopandas as gpd
from scipy.interpolate import griddata
import rioxarray
from shapely.geometry import mapping
import xarray as xr
import os

In [8]:
# -- IMPORTING DATA
# File Paths
country_boundary_file   = '../data/raw/external/GADM41_WORLD.gpkg'
dates_file              = '../data/raw/external/PAI_dates.xlsx'
gridded_dates_file      = '../data/preprocessed/external/monsoon_dates.nc'

# Check if the files exist [if preprocessed files exist abort the process]
if not os.path.exists(country_boundary_file):
    raise FileNotFoundError(f"{country_boundary_file} not found.")
if not os.path.exists(dates_file):
    raise FileNotFoundError(f"{dates_file} not found.")
if os.path.exists(gridded_dates_file):
    print(f"{dates_file} found. The file will be overwritten!")

# Load country boundary data
country_boundary = gpd.read_file(country_boundary_file, layer='ADM_1')

# Load date data
dates_tab = pd.read_excel(dates_file)

# Select relevant columns
dates_tab = dates_tab[['lat', 'lon', 'ons_num', 'wit_num']]

../data/raw/external/PAI_dates.xlsx found. The file will be overwritten!


In [9]:
# -- GRIDDING DATASET
# Extract longitude, latitude, onset, and withdrawal dates as numpy arrays
lon = dates_tab['lon'].to_numpy()
lat = dates_tab['lat'].to_numpy()
onset = dates_tab['ons_num'].to_numpy()
withdrawal = dates_tab['wit_num'].to_numpy()

# Create a list of (lon, lat) points
points = list(zip(lon, lat))

# Define the new grid resolution
lon_fin = np.arange(65, 100.25, 0.25)
lat_fin = np.arange(5, 40.25, 0.25)

# Create a meshgrid for the new grid
gridX, gridY = np.meshgrid(lon_fin, lat_fin)

# Interpolate onset and withdrawal dates using cubic method
regrid_ons = griddata(points, onset, (gridX, gridY), method='cubic')
regrid_wit = griddata(points, withdrawal, (gridX, gridY), method='cubic')

# Stack the onset and withdrawal dates into a single array
dates = np.stack((regrid_ons, regrid_wit), axis=2)

In [11]:
# -- CREATING XARRAY DATASET AND SAVING
# Create an xarray Dataset
ds = xr.Dataset(
    data_vars=dict(
        date=(["lat", "lon", "season_idx"], dates),
    ),
    coords=dict(
        lon=lon_fin,
        lat=lat_fin,
        season_idx=['onset', 'withdrawal']
    ),
     attrs=dict(
        Description="Interpolated onset and withdrawal dates of the monsoon season across India.",
        History=(
            "1. Country boundaries were loaded from GADM41_IND.gpkg.\n"
            "2. Monsoon onset and withdrawal dates were extracted from PAI_dates.xlsx.\n"
            "3. The data was gridded using a 0.25-degree resolution.\n"
            "4. Cubic interpolation was performed to estimate onset and withdrawal dates on the new grid.\n"
            "5. The interpolated data was stored in an xarray Dataset."
        )
    )
    )

# Masking with country shapefile
ds.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
ds.rio.write_crs("epsg:4326", inplace=True)
ds_rast = ds.rio.clip(country_boundary.geometry.apply(mapping), country_boundary.crs, drop=True)

# Save xarray Dataset to a NetCDF file (optional)
ds_rast.to_netcdf(gridded_dates_file)

ds_rast

<xarray.Dataset>
Dimensions:      (lon: 141, lat: 141, season_idx: 2)
Coordinates:
  * lon          (lon) float64 65.0 65.25 65.5 65.75 ... 99.25 99.5 99.75 100.0
  * lat          (lat) float64 5.0 5.25 5.5 5.75 6.0 ... 39.25 39.5 39.75 40.0
  * season_idx   (season_idx) <U10 'onset' 'withdrawal'
    spatial_ref  int64 0
Data variables:
    date         (lat, lon, season_idx) float64 nan nan nan nan ... nan nan nan
Attributes:
    Description:  Interpolated onset and withdrawal dates of the monsoon seas...
    History:      1. Country boundaries were loaded from GADM41_IND.gpkg.\n2....